# Imports

In [2]:
# Packages
import pandas as pd

# Helper functions
from ipynb.fs.full.functions import *

# Other notebooks for combining code
from ipynb.fs.full.semrush_call import execute_call
from ipynb.fs.full.text_division_and_styling import LoopGetHeaderCountFromURL, LoopGetListsCountFromURL
from ipynb.fs.full.semrush_beoordeling_concurrentieniveau import getKeywordDifficulty 
from ipynb.fs.full.InboundOutboundScraper import loop_get_all_website_links
from ipynb.fs.full.SemRush_InboundOutbound_Backlink import get_all_website_links_semrush
from ipynb.fs.full.img_scraper import *

Deze call is al een keer uitgevoerd, we gaan verder met de bestaande resultaten.
Dit bestand is laatst gewijzigd op: Tue May 11 12:42:44 2021


FileNotFoundError: [Errno 2] No such file or directory: 'calls\\keywordDifficulty-2ec327fb55baedc6a97442bf8aea462a1f8c84e45573ebb0b966c45da7afc517.csv'

# Executing of functions
## Variables for keyword search

In [3]:
keyword = 'bouwmaterialen' # What keyword do you want to call for?
result_count = '5' # How many keyword results do you want to gather data for?

# Do you want to overwrite the SEMrush keyword call if it has already been executed before?
# This is to save on SEMrush token usage
potential_overwrite = False

# Do you want to save the result of the end dataframe?
save = True

## Creating dataframes

In [4]:
def ExecuteAllCalls():
    # Define variables for the SEMrush call and gather keyword data
    export_columns = 'Dn,Ur'
    endpoint_url = 'https://api.semrush.com/'
    main_dataframe = execute_call(keyword, result_count, export_columns, endpoint_url, potential_overwrite)
    
    # Add used keyword to the dataframe
    keyword_list = []
    keyword_list += int(result_count) * [keyword]
    main_dataframe['Keyword'] = keyword_list
    
    # Executing functions from other notebooks to gather data
    print("  0% - Executing getKeywordDifficulty")
    KeywordDifficultyFunction = getKeywordDifficulty(keyword, potential_overwrite)
    KeywordDifficultyList = []
    KeywordDifficultyList += int(result_count) * [KeywordDifficultyFunction['Kd'][0]]
    main_dataframe['Kd'] = KeywordDifficultyList
    
    print(" 15% - Executing LoopGetHeaderCountFromURL")
    main_dataframe['HeaderCount'] = LoopGetHeaderCountFromURL(main_dataframe)
    print(" 30% - Executing LoopGetListsCountFromURL")
    main_dataframe['ListsCount'] = LoopGetListsCountFromURL(main_dataframe)
    print(" 45% - Executing loop_get_all_website_links")
    website_links = loop_get_all_website_links(main_dataframe)
    main_dataframe['InternalLinksSoup'] = website_links[0]
    main_dataframe['ExternalLinksSoup'] = website_links[1]
    print(" 60% - Executing get_all_website_links_semrush")
    main_dataframe['InternalLinksSEMrush'] = get_all_website_links_semrush(main_dataframe, 'internal', potential_overwrite)
    main_dataframe['ExternalLinksSEMrush'] = get_all_website_links_semrush(main_dataframe, 'external', potential_overwrite)
    main_dataframe['BacklinksSEMrush'] = get_all_website_links_semrush(main_dataframe, 'backlink', potential_overwrite)
    print(" 75% - Executing createDF")    
    main_dataframe = main_dataframe.merge(createDF(main_dataframe).drop('Domain', axis=1), how='inner', left_on='Ur', right_on='url')
    
    # Set correct index, potentially save the output and return the dataframe
    main_dataframe = main_dataframe.set_index('Number').drop('url', axis=1) # Drop URL because it stay in there from the merge
    main_dataframe_hash = hash_variable(main_dataframe)
    if save == True:
        main_dataframe.to_csv(f"calls\dataframe-{main_dataframe_hash}.csv")
    print("100% - Done")
    return main_dataframe

In [5]:
main_dataframe = ExecuteAllCalls()
main_dataframe

Deze call is al een keer uitgevoerd, we gaan verder met de bestaande resultaten.
Dit bestand is laatst gewijzigd op: Fri Apr 23 09:28:02 2021


FileNotFoundError: [Errno 2] No such file or directory: 'calls\\call-21a8a24a193bbb3dc3a5537d0dacf45c603bb1a53cb266ddf3dbf8101e68010d.csv'